# Train YOLOv8 on a Custom Dataset & Export for Deployment Windows, Android, or iOS - Erik Elcsics

This notebook will guide you through training a YOLOv8 model on your own dataset, step by step. It’s beginner-friendly and ends with exporting the trained model for use on Windows, Android, or iOS. - Erik Elcsics

In [ ]:
# 1. Install YOLOv8 (Ultralytics)
# This installs the Ultralytics package, which includes everything you need to train and use YOLOv8
!pip install -q ultralytics
from ultralytics import YOLO

In [ ]:
# 2. Prepare Dataset in YOLO Format

"""
Your dataset must follow a specific folder structure:

/dataset
  /images
    /train  -> training images (e.g., image1.jpg)
    /val    -> validation images
  /labels
    /train  -> corresponding labels (e.g., image1.txt)
    /val

Each .txt file should contain lines like:
<class_id> <x_center> <y_center> <width> <height> (values normalized between 0 and 1)
"""

import os

# Create folders if they don't already exist
os.makedirs('data/images/train', exist_ok=True)
os.makedirs('data/images/val', exist_ok=True)
os.makedirs('data/labels/train', exist_ok=True)
os.makedirs('data/labels/val', exist_ok=True)

### 3. Create the dataset.yaml Config File

YOLOv8 needs a YAML file that tells it where to find the training/validation images and what the class names are.

Save this as `data.yaml` in the root of your project:

In [ ]:
# This is just to show the YAML format – you need to save it as data.yaml manually or using code.
print("""train: data/images/train
val: data/images/val

nc: 2
names: ['cat', 'dog']
""")

In [ ]:
#  4. Train the YOLOv8 Model

# YOLOv8n is the smallest model and fastest to train (good for testing and limited hardware).
model = YOLO("yolov8n.pt")  # Options: yolov8n.pt, yolov8s.pt, etc.

# Start training
results = model.train(
    data="data.yaml",   # Path to the dataset config file
    epochs=50,          # Number of times the model will see the entire dataset
    imgsz=640,          # Image size (can affect accuracy and speed)
    batch=16,           # How many images to train on at once
    name="custom_yolov8_model"  # Folder where results will be saved
)

In [ ]:
#  5. Visualize Training Results
# This will display charts showing how well the model trained (accuracy, loss, etc.)
results.plot()

In [ ]:
#  6. Run Inference on Test Images
# Now use your trained model to make predictions on validation images
model = YOLO("runs/detect/custom_yolov8_model/weights/best.pt")
results = model.predict(source="data/images/val", save=True, conf=0.25)  # conf is confidence threshold

In [ ]:
#  7. View Predictions Inline
# This displays the prediction results from your validation/test images
import glob
from IPython.display import Image, display

predicted_images = glob.glob('runs/detect/predict/*.jpg')
for img_path in predicted_images:
    display(Image(filename=img_path))

In [ ]:
#  8. Export Model for Deployment
# Export to formats suitable for Windows (ONNX), Android (TFLite), and iOS (CoreML)
model.export(format="onnx")      # Windows PC or general desktop use
model.export(format="tflite")    # Android
model.export(format="coreml")    # iPhone/iOS

In [ ]:
#  Optional: Download exported models from Colab to your local machine
from google.colab import files

files.download("runs/detect/custom_yolov8_model/weights/best.onnx")
files.download("runs/detect/custom_yolov8_model/weights/best.tflite")
files.download("runs/detect/custom_yolov8_model/weights/best.mlmodel")